# OWNER LEVEL

HPD 317 Phone Codes

In [23]:
import pandas as pd
import numpy as np

path = "C:\\Users\\dmehri\\Documents\\DATA\\Building Census\\Columbia Data\\illegal conv\\"

df = pd.read_csv(path + "Property BIN and Phone Number with Codes HPD COLUMBIA.csv")

df["Building Count"] = 1

df["BIN"] = df["BIN"].astype(str).map(str.strip)

print "len before drop", len(df)
df = df[df["NumberDigits"] == 10]
print len(df)
df["PhoneCode"] = df["PhoneCode"].astype(str).map(str.strip)
df = df[df["PhoneCode"] != "301611"].reset_index(drop=True)
print len(df)

uniqPhoneCode = set(df["PhoneCode"].tolist())

print len(uniqPhoneCode)


len before drop 622992
499278
499278
244132


In [24]:
df.head(1)

,BIN,BBL,NumberDigits,AreaCode,PhoneCode,Building Count
0,1000005,1000047501,10,201,66088,1


Illegal Convers Complaints

In [21]:

def FinalComplaintDisposition(d):
    print "len of datframe", len(d)
    complaints = sorted(list(set(d["Complaint Number"].tolist()) ) )
    print "number of unique complaints", len(complaints)
    
    all_data = pd.DataFrame()
    
    d['Inspection Date'] = pd.to_datetime(d['Inspection Date'])

    for i in range(0, len(complaints)):
        if i%3000 == 0:
            print i
        d2 = d[d["Complaint Number"] == complaints[i]]
        
        #sort to get the df with the latest dist
        d2 = d2.sort_values(by = 'Inspection Date', ascending=False).reset_index(drop=True)

        d2["Index"] = d2.index
        
        #subset the dataframe to get the latest date
        d2 = d2[d2["Index"] < 1]
        
        #print i
        #print d2
        #print "*****************"
        
        #add the dataframes
        all_data = all_data.append(d2,ignore_index=False)
    
    #final dataframe
    all_data = all_data.reset_index(drop=True)
    
    print "len of final dataframe", len(all_data)


    return all_data

    #dd.to_csv(path + "test.csv", index=False)

#x = FinalComplaintDisposition(di)

    

In [73]:
x.head(1)

,Complaint Number,Status,Date Entered,House Number,ZIP Code,House Street,BIN,Community Board,Special District,Complaint Category,Unit,Disposition Date,Disposition Code,Inspection Date,DOBRunDate,Index
0,1440433,CLOSED,2017-01-03,132,10027,WEST 129 STREET,1057957,110,,45,MAN.,03/10/2017,A8,2017-03-09,11/07/2018 12:00:00 AM,0


In [74]:
#di = pd.read_csv(path + "DOB_Complaints__Illegal_Conversions from 1_2017.csv",low_memory=False )
#di = pd.read_csv(path + "DOB_Complaints_Received_Illegal_Conversions.csv",low_memory=False )
pathb = "C:\\Users\\dmehri\\Documents\\DATA\\Building Census\\Columbia Data\\"

di = pd.read_csv(pathb + "DOB_Complaints_Received.csv",low_memory=False )

print "len of all complaints", len(di)

di["Complaint Category"] = di["Complaint Category"].astype(str).map(str.strip)

print "get only illegal conversion complaints, 45, 31, 71"

illegconv = ["45", "31", "71"]

di = di[di['Complaint Category'].isin(illegconv)].reset_index(drop=True)

print "len after getting only ics", len(di)


print "get only tru complaints, drop I1, I2"

di["Disposition Code"] = di["Disposition Code"].astype(str).map(str.strip)

ilist = ["I1", "I2"]

di = di[~di['Disposition Code'].isin(ilist)].reset_index(drop=True)

print "len after drop I1, I2", len(di)


di['Date Entered'] = pd.to_datetime(di['Date Entered'])

di["BIN"] = di["BIN"].astype(str).map(str.strip)

di = di.sort_values(by = 'Date Entered', ascending=True).reset_index(drop=True)

start_time = str(1) + '-' + str(1) + '-' + str(2017)
mask = di['Date Entered'] >= start_time
print "len before drop", len(di)
di = di.loc[mask].reset_index(drop=True)
print "len after drop", len(di)

di = FinalComplaintDisposition(di)

di2 = di[["BIN"]]

di2["Count"] = 1
print "len di2", len(di2)
di2G = di2.groupby(['BIN']).sum()
di2G = di2G.add_suffix('').reset_index()

di2Dic =  di2G.set_index('BIN')['Count'].to_dict()
print "len di2Dic", len(di2Dic)

print "Map to df"

df["#ICComplaints"] = df["BIN"].map(di2Dic)
df["#ICComplaints"] = df["#ICComplaints"].fillna(0)



len of all complaints 2569490
get only illegal conversion complaints, 45, 31, 71
len after getting only ics 489177
get only tru complaints, drop I1, I2
len after drop I1, I2 376556
len before drop 376556
len after drop 97118
len of datframe 97118
number of unique complaints 57398
0
3000
6000
9000
12000
15000
18000
21000
24000
27000
30000
33000
36000
39000
42000
45000
48000
51000
54000
57000
len di2 57398
len di2Dic 36000
Map to df


In [75]:
df.head(1)

,BIN,BBL,NumberDigits,AreaCode,PhoneCode,Building Count,#ICComplaints,#ICVios,#ICNoAccess,NA
0,1000005,1000047501,10,201,66088,1,0.0,0.0,0.0,1.0


Illegal Conversion Violations and No Access

In [83]:
pd.set_option('chained_assignment', None)


dd = di.copy()

print "len of all complaints", len(dd)


dd2 = dd.copy()

dd2 = dd2[["BIN","Status","Date Entered", "Disposition Code"]]

dd2["Disposition Code"] = dd2["Disposition Code"].astype(str).map(str.strip)

#codeCount = dd2[["Disposition Code"]]

#codeCount['Disposition Code'] = codeCount['Disposition Code'].str.replace(' ', 'nan')
#codes = set(dd2["Disposition Code"].tolist())

violist = ["A1", "A2", "A3", "A4", "A5", "A6", "A7", "A8", "A9",
          "B1", "B2", "H3", "H4", "H5", "L1", "L3", "Q1", "Q2",
          "Q3", "Q4", "Q5", "RU", "W1"]

noaccess = ["C1", "C2", "C3", "C4", "C5", "C6", "C7", "C8"]

#noncomplaint = ["I1", "I2"]

#codeCount["Count"] = 1
#codeCount = codeCount.groupby(['Disposition Code']).sum()
#codeCount = codeCount.add_suffix('').reset_index()

print len(dd2)

dd2["Violation"] = 0
dd2["No Access"] = 0
#dd2["Complaint Unsubstantiated"] = 0
count = 0
countnoaccess = 0
#countnoncomplaint = 0
for i in range(0, len(dd2)):
    if i%8000==0:
        print i
    if dd2["Disposition Code"][i] in violist:
        dd2["Violation"][i] = 1
        count+=1
    if dd2["Disposition Code"][i] in noaccess:
        dd2["No Access"][i] = 1
        countnoaccess+=1
    #if dd2["Disposition Code"][i] in noncomplaint:
    #    dd2["Complaint Unsubstantiated"][i] = 1
    #    countnoncomplaint+=1

dd2G = dd2[["BIN","Violation", "No Access" ]]
    
dd2G = dd2G.groupby(['BIN']).sum()
dd2G = dd2G.add_suffix('').reset_index()

#dd2G = dd2G.sort_values(by = 'Violation', ascending=False).reset_index(drop=True)

dd2DicVio = dd2G.set_index('BIN')['Violation'].to_dict()

dd2DicNoAccess = dd2G.set_index('BIN')['No Access'].to_dict()

df["#ICVios"] = df["BIN"].map(dd2DicVio)
df["#ICVios"] = df["#ICVios"].fillna(0)

df["#ICNoAccess"] = df["BIN"].map(dd2DicNoAccess)
df["#ICNoAccess"] = df["#ICNoAccess"].fillna(0)

print
print "total number of dispositions", len(dd2)
print "number of violations", count
print "number no access", countnoaccess
#print "number non complaints", countnoncomplaint
    
#codes



57398
0
8000
16000
24000
32000
40000
48000
56000

total number of dispositions 57398
number of violations 6752
number no access 36958


In [82]:
#di.to_csv(path + "DOB_Complaints_Received_Final_Disposition.csv", index=False)

In [84]:
df.to_csv(path + "test.csv", index=False)

In [34]:
path

'C:\\Users\\dmehri\\Documents\\DATA\\Building Census\\Columbia Data\\illegal conv\\'

In [ ]:
#df.to_csv(path + "test.csv", index=False)

HPD NOT AVAILABLE

In [85]:
pd.set_option('chained_assignment', None)

dna = pd.read_csv(path + "Comprehensive 1- 317K HPD List.csv",low_memory=False )

dna["BIN"] = dna["BIN"].astype(str).map(str.strip)
dna["HPD Classifications"] = dna["HPD Classifications"].astype(str).map(str.strip).map(str.upper)

na = dna[dna["HPD Classifications"] == "NOT AVAILABLE"]

print len(na)

na["Count"] = 1

naDic =  na.set_index('BIN')['Count'].to_dict()

df["NA"] = df["BIN"].map(naDic)
df["NA"] = df["NA"].fillna(0)


123129


In [36]:
df.head(3)

,BIN,BBL,NumberDigits,AreaCode,PhoneCode,Building Count,#ICComplaints,#ICVios,#ICNoAccess,NA
0,1000005,1000047501,10,201,66088,1,0.0,0.0,0.0,1.0
1,1000005,1000047501,10,201,345745,1,0.0,0.0,0.0,1.0
2,1000005,1000047501,10,201,117369,1,0.0,0.0,0.0,1.0


DEVELOP RISK ANALYTICS OR BUILDING OWNERS

In [ ]:
#df["IllegConv&NA"] = 0
#for i in range(0, len(df)):
#    if (df["IllegConvComplaints"][i] == 1 and df["NA"][i] == 1):
#        df["IllegConv&NA"][i] = 1
        

In [37]:
df.head(1)

,BIN,BBL,NumberDigits,AreaCode,PhoneCode,Building Count,#ICComplaints,#ICVios,#ICNoAccess,NA
0,1000005,1000047501,10,201,66088,1,0.0,0.0,0.0,1.0


GROUPBY AT THE PHONE CODE LEVEL

In [90]:
df2 = df[["PhoneCode", "Building Count", "#ICComplaints","#ICVios",  
          "#ICNoAccess",  "NA"]]


df2 = df2.groupby(['PhoneCode']).sum()

df2 = df2.add_suffix('').reset_index()

df2 = df2.sort_values(by = 'Building Count', ascending=False).reset_index(drop=True)

df2["#ICComplaintsNoAccess"] = df2["#ICComplaints"] + df2["#ICNoAccess"]

"""
df2["%IllegConvComplaints"] = (df2["IllegConvComplaints"]/df2["Building Count"] )*100

df2["%IllegConvVios"] = (df2["IllegConvVios"]/df2["Building Count"] )*100

df2["%NA"] = (df2["NA"]/df2["Building Count"] )*100
#df2["%IllegConv&NA"] = (df2["IllegConv&NA"]/df2["Building Count"] ) *100

df2["%IllegConvComplaints"] = df2["%IllegConvComplaints"].round(1)
df2["%IllegConvVios"] = df2["%IllegConvVios"].round(1)
df2["%NA"] = df2["%NA"].round(1)
#df2["%IllegConv&NA"] = df2["%IllegConv&NA"].round(1)

"""

'\ndf2["%IllegConvComplaints"] = (df2["IllegConvComplaints"]/df2["Building Count"] )*100\n\ndf2["%IllegConvVios"] = (df2["IllegConvVios"]/df2["Building Count"] )*100\n\ndf2["%NA"] = (df2["NA"]/df2["Building Count"] )*100\n#df2["%IllegConv&NA"] = (df2["IllegConv&NA"]/df2["Building Count"] ) *100\n\ndf2["%IllegConvComplaints"] = df2["%IllegConvComplaints"].round(1)\ndf2["%IllegConvVios"] = df2["%IllegConvVios"].round(1)\ndf2["%NA"] = df2["%NA"].round(1)\n#df2["%IllegConv&NA"] = df2["%IllegConv&NA"].round(1)\n\n'

In [92]:
path

'C:\\Users\\dmehri\\Documents\\DATA\\Building Census\\Columbia Data\\illegal conv\\'

In [93]:
df2.to_csv(path + "test.csv", index=False)

Count Number of Unique Owners Who Own More than 1 Building

In [ ]:
dfunique = df2[df2["Building Count"] > 1]

uniqPhoneCode = set(dfunique["PhoneCode"].tolist())

print len(uniqPhoneCode)


In [ ]:
df2.head(1)

In [ ]:
df2.head(1)

In [94]:
#print "Outputing data"

print "Only include owners who own 5 building or more"
print "len before drop", len(df2)
df3 = df2[df2["Building Count"] >= 5].reset_index(drop=True)
print "len after drop", len(df3)

print "create a categorical variable for building count"

df3["Building Count Category"] = ""

for i in range(0, len(df3)):
    if df3["Building Count"][i] <= 10:
        df3["Building Count Category"][i] = "5_10"
    if (df3["Building Count"][i] > 10 and df3["Building Count"][i] <= 20):
        df3["Building Count Category"][i] = "11_20"
    if (df3["Building Count"][i] > 20 and df3["Building Count"][i] <= 30):
        df3["Building Count Category"][i] = "21_30"
    if (df3["Building Count"][i] > 30 and df3["Building Count"][i] <= 40):
        df3["Building Count Category"][i] = "31_40"
    if (df3["Building Count"][i] > 40 and df3["Building Count"][i] <= 50):
        df3["Building Count Category"][i] = "41_50"
    if (df3["Building Count"][i] > 50 and df3["Building Count"][i] <= 60):
        df3["Building Count Category"][i] = "51_60"
    if (df3["Building Count"][i] > 60 and df3["Building Count"][i] <= 70):
        df3["Building Count Category"][i] = "61_70"
    if (df3["Building Count"][i] > 70 and df3["Building Count"][i] <= 80):
        df3["Building Count Category"][i] = "71_80"
    if (df3["Building Count"][i] > 80 and df3["Building Count"][i] <= 90):
        df3["Building Count Category"][i] = "81_90"
    if (df3["Building Count"][i] > 90 and df3["Building Count"][i] <= 100):
        df3["Building Count Category"][i] = "91_100"
    if (df3["Building Count"][i] > 100):
        df3["Building Count Category"][i] = ">100"
    


Only include owners who own 5 building or more
len before drop 244132
len after drop 10717
create a categorical variable for building count


In [95]:
df3.head(1)

,PhoneCode,Building Count,#ICComplaints,#ICVios,#ICNoAccess,NA,#ICComplaintsNoAccess,Building Count Category
0,162639,2247,194.0,49.0,56.0,362.0,250.0,>100


DEVELOP RISK SCORE FOR ILLEGAL CONVERSION COMPLAINTS

In [97]:
#N = np.log(np.exp(1000) )
#print N

print "calculate ln of building count"

df3["ln_build_count"] = 0.0

df3["Risk Score IllegConv Complaints"] = 0.0

df3["Risk Category IllegConv Complaints"] = ""

#df3["ln_build_count"] = df3["Building Count"].log()

for i in range(0, len(df3)):
    #print df3["Building Count"][i]
    df3["ln_build_count"][i] = np.log(df3["Building Count"][i])
    
    ln_bc = df3["ln_build_count"][i]
    
    df3["Risk Score IllegConv Complaints"][i] = df3["#ICComplaints"][i]/(ln_bc**2)
    
    if df3["Risk Score IllegConv Complaints"][i] > 1:
        df3["Risk Score IllegConv Complaints"][i] = 1
    
    df3["Risk Score IllegConv Complaints"][i] = (df3["Risk Score IllegConv Complaints"][i]*10).round(0)
    
    if df3["Risk Score IllegConv Complaints"][i] >= 8:
        df3["Risk Category IllegConv Complaints"][i] = "HIGH"
        
    elif (df3["Risk Score IllegConv Complaints"][i] >= 5 and df3["Risk Score IllegConv Complaints"][i] < 8):
        df3["Risk Category IllegConv Complaints"][i] = "MEDIUM"

    else:
        df3["Risk Category IllegConv Complaints"][i] = "LOW"


df3["Risk Score IllegConv Complaints"] = df3["Risk Score IllegConv Complaints"].astype(int)


calculate ln of building count


In [98]:
df3.dtypes

PhoneCode                              object
Building Count                          int64
#ICComplaints                         float64
#ICVios                               float64
#ICNoAccess                           float64
NA                                    float64
#ICComplaintsNoAccess                 float64
Building Count Category                object
ln_build_count                        float64
Risk Score IllegConv Complaints         int32
Risk Category IllegConv Complaints     object
dtype: object

RISK SOCRE FOR ILLEGAL CONVERSION VIOLATIONS

In [99]:

df3["Risk Score IllegConv Violations"] = 0.0

df3["Risk Category IllegConv Violations"] = ""

#df3["ln_build_count"] = df3["Building Count"].log()

for i in range(0, len(df3)):
    #print df3["Building Count"][i]
    
    ln_bc = df3["ln_build_count"][i]
    
    df3["Risk Score IllegConv Violations"][i] = df3["#ICVios"][i]/(ln_bc**2)
    
    if df3["Risk Score IllegConv Violations"][i] > 1:
        df3["Risk Score IllegConv Violations"][i] = 1
    
    df3["Risk Score IllegConv Violations"][i] = (df3["Risk Score IllegConv Violations"][i]*10).round(0)
    
    if df3["Risk Score IllegConv Violations"][i] >= 8:
        df3["Risk Category IllegConv Violations"][i] = "HIGH"
        
    elif (df3["Risk Score IllegConv Violations"][i] >= 5 and df3["Risk Score IllegConv Violations"][i] < 8):
        df3["Risk Category IllegConv Violations"][i] = "MEDIUM"

    else:
        df3["Risk Category IllegConv Violations"][i] = "LOW"


df3["Risk Score IllegConv Violations"] = df3["Risk Score IllegConv Violations"].astype(int)


In [ ]:
#df3.to_csv(path + "IllegConv Ownership Risk Score.csv", index=False)

In [100]:
df3.head(1)

,PhoneCode,Building Count,#ICComplaints,#ICVios,#ICNoAccess,NA,#ICComplaintsNoAccess,Building Count Category,ln_build_count,Risk Score IllegConv Complaints,Risk Category IllegConv Complaints,Risk Score IllegConv Violations,Risk Category IllegConv Violations
0,162639,2247,194.0,49.0,56.0,362.0,250.0,>100,7.717351,10,HIGH,8,HIGH


Count Number of High Risk Illegal Conversion by Building Count Category

In [ ]:
"""
buildCountCat = df3[df3["Risk Score IllegConv"] > 7]

buildCountCat = buildCountCat[["Building Count Category"]]
buildCountCat["Count"] = 1
buildCountCat=buildCountCat.groupby('Building Count Category').sum()
buildCountCat = buildCountCat.add_suffix('').reset_index()
buildCountCat = buildCountCat.sort_values(by = 'Count', ascending=False).reset_index(drop=True)

buildCountCat
"""


RISK SCORE FOR NAs

In [101]:
N = np.log(np.exp(1000) )
#print N

print "Calculate Risk Scores for NAs"


df3["Risk Score NA"] = 0.0

df3["Risk Category NA"] = ""

#df3["ln_build_count"] = df3["Building Count"].log()

for i in range(0, len(df3)):
    #print df3["Building Count"][i]
    #df3["ln_build_count"][i] = np.log(df3["Building Count"][i])
    
    ln_bc = df3["ln_build_count"][i]
    
    df3["Risk Score NA"][i] = df3["NA"][i]/(ln_bc**2)
    
    if df3["Risk Score NA"][i] > 1:
        df3["Risk Score NA"][i] = 1
    
    df3["Risk Score NA"][i] = (df3["Risk Score NA"][i]*10).round(0)
    
    if df3["Risk Score NA"][i] >= 8:
        df3["Risk Category NA"][i] = "HIGH"
        
    elif (df3["Risk Score NA"][i] >= 5 and df3["Risk Score NA"][i] < 8):
        df3["Risk Category NA"][i] = "MEDIUM"

    else:
        df3["Risk Category NA"][i] = "LOW"


df3["Risk Score NA"] = df3["Risk Score NA"].astype(int)

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: RuntimeWarning: overflow encountered in exp
  if __name__ == '__main__':


Calculate Risk Scores for NAs


In [103]:
df3.dtypes

PhoneCode                              object
Building Count                          int64
#ICComplaints                         float64
#ICVios                               float64
#ICNoAccess                           float64
NA                                    float64
#ICComplaintsNoAccess                 float64
Building Count Category                object
ln_build_count                        float64
Risk Score IllegConv Complaints         int32
Risk Category IllegConv Complaints     object
Risk Score IllegConv Violations         int32
Risk Category IllegConv Violations     object
Risk Score NA                           int32
Risk Category NA                       object
dtype: object

RISK SCORE COMPLAINTS AND NO ACCESS

In [105]:
print "Calculate Risk Scores for Complaint/No Access"

df3["Risk Score CompNoAccess"] = 0.0

df3["Risk Category CompNoAccess"] = ""

#df3["ln_build_count"] = df3["Building Count"].log()

for i in range(0, len(df3)):
    #print df3["Building Count"][i]
    #df3["ln_build_count"][i] = np.log(df3["Building Count"][i])
    
    ln_bc = df3["ln_build_count"][i]
    
    df3["Risk Score CompNoAccess"][i] = df3["#ICComplaintsNoAccess"][i]/(ln_bc**2)
    
    if df3["Risk Score CompNoAccess"][i] > 1:
        df3["Risk Score CompNoAccess"][i] = 1
    
    df3["Risk Score CompNoAccess"][i] = (df3["Risk Score CompNoAccess"][i]*10).round(0)
    
    if df3["Risk Score CompNoAccess"][i] >= 8:
        df3["Risk Category CompNoAccess"][i] = "HIGH"
        
    elif (df3["Risk Score CompNoAccess"][i] >= 5 and df3["Risk Score CompNoAccess"][i] < 8):
        df3["Risk Category CompNoAccess"][i] = "MEDIUM"

    else:
        df3["Risk Category CompNoAccess"][i] = "LOW"


df3["Risk Score CompNoAccess"] = df3["Risk Score CompNoAccess"].astype(int)

Calculate Risk Scores for Complaint/No Access


In [107]:
df3.to_csv(path + "test.csv", index=False)

In [106]:
df3.head(1)

,PhoneCode,Building Count,#ICComplaints,#ICVios,#ICNoAccess,NA,#ICComplaintsNoAccess,Building Count Category,ln_build_count,Risk Score IllegConv Complaints,Risk Category IllegConv Complaints,Risk Score IllegConv Violations,Risk Category IllegConv Violations,Risk Score NA,Risk Category NA,Risk Score CompNoAccess,Risk Category CompNoAccess
0,162639,2247,194.0,49.0,56.0,362.0,250.0,>100,7.717351,10,HIGH,8,HIGH,10,HIGH,10,HIGH


### FINAL HIGH RISK SCORE

In [108]:
df3.dtypes

PhoneCode                              object
Building Count                          int64
#ICComplaints                         float64
#ICVios                               float64
#ICNoAccess                           float64
NA                                    float64
#ICComplaintsNoAccess                 float64
Building Count Category                object
ln_build_count                        float64
Risk Score IllegConv Complaints         int32
Risk Category IllegConv Complaints     object
Risk Score IllegConv Violations         int32
Risk Category IllegConv Violations     object
Risk Score NA                           int32
Risk Category NA                       object
Risk Score CompNoAccess                 int32
Risk Category CompNoAccess             object
dtype: object

In [109]:
df3["Final Score"] = (1.3*df3["Risk Score CompNoAccess"] + 
                      0.7*df3["Risk Score NA"])

df3["Final Score"] = (df3["Final Score"]/20.0)*10

df3["Final Risk Category"] = ""

for i in range(0, len(df3)):
    if df3["Final Score"][i] >= 8:
        df3["Final Risk Category"][i] = "HIGH"
    elif df3["Final Score"][i] < 5:
        df3["Final Risk Category"][i] = "LOW"
    else:
        df3["Final Risk Category"][i] = "MEDIUM"
       

                      


In [ ]:
len(df3)

In [110]:
df3.head(5)

,PhoneCode,Building Count,#ICComplaints,#ICVios,#ICNoAccess,NA,#ICComplaintsNoAccess,Building Count Category,ln_build_count,Risk Score IllegConv Complaints,Risk Category IllegConv Complaints,Risk Score IllegConv Violations,Risk Category IllegConv Violations,Risk Score NA,Risk Category NA,Risk Score CompNoAccess,Risk Category CompNoAccess,Final Score,Final Risk Category
0,162639,2247,194.0,49.0,56.0,362.0,250.0,>100,7.717351,10,HIGH,8,HIGH,10,HIGH,10,HIGH,10.0,HIGH
1,262705,2028,149.0,30.0,78.0,304.0,227.0,>100,7.614805,10,HIGH,5,MEDIUM,10,HIGH,10,HIGH,10.0,HIGH
2,438887,1804,130.0,31.0,43.0,281.0,173.0,>100,7.497762,10,HIGH,6,MEDIUM,10,HIGH,10,HIGH,10.0,HIGH
3,20767,1498,45.0,8.0,17.0,128.0,62.0,>100,7.311886,8,HIGH,1,LOW,10,HIGH,10,HIGH,10.0,HIGH
4,295187,1368,61.0,14.0,35.0,200.0,96.0,>100,7.221105,10,HIGH,3,LOW,10,HIGH,10,HIGH,10.0,HIGH


In [111]:
df3.to_csv(path + "IllegConv Ownership Risk Score 3.csv", index=False)


In [ ]:
df3.head(1)

# BUILDING LEVEL

USE RISK SCORE TO DEVELOP LIST OF BUILDINGS UNKNOWN TO DOB

In [122]:
path = "C:\\Users\\dmehri\\Documents\\DATA\\Building Census\\Columbia Data\\illegal conv\\"

dr = pd.read_csv(path + "IllegConv Ownership Risk Score 3.csv")

print len(dr)

dr = dr[dr["Final Risk Category"] == "HIGH"].reset_index(drop=True)

dr["PhoneCode"] = dr["PhoneCode"].astype(str).map(str.strip)

print len(dr)

highphonecode = list(set(dr["PhoneCode"].tolist() ))


df = pd.read_csv(path + "Property BIN and Phone Number with Codes HPD COLUMBIA.csv")

df["PhoneCode"] = df["PhoneCode"].astype(str).map(str.strip)


df.head(1)

print "len befor subsetting for high risk phone codes"
print len(df)

df = df[df['PhoneCode'].isin(highphonecode)].reset_index(drop=True)
print len(df)

print "drop dupes"
df = df.drop_duplicates(['BIN']).reset_index(drop=True)

df["BIN"] = df["BIN"].astype(str).map(str.strip)

binlist = df["BIN"].tolist()


print len(df)
print "import columbia regression dataset, has all info about complaints"

dc = pd.read_csv(path + "Building Censes Illeg Conv Columbia Regression.csv")
dc["BIN"] = dc["BIN"].astype(str).map(str.strip)
dc = dc[dc['BIN'].isin(binlist)].reset_index(drop=True)
print len(dc)



#di = pd.read_csv(path + "DOB_Complaints_Received_Final_Disposition_with_I1-2.csv")

#di["BIN"] = di["BIN"].astype(str).map(str.strip)
#df["BIN"] = df["BIN"].astype(str).map(str.strip)

#di = di[["BIN"]]
#di["Complaint"] = 1

#diDic = di.set_index('BIN')['Complaint'].to_dict()

#df["Complaint"] = df["BIN"].map(diDic)
#df["Complaint"] = df["Complaint"].fillna(0)

#df2 = df[df["Complaint"] == 0].reset_index(drop=True)

#print "len of high risk buildings", len(df2)


10717
616
len befor subsetting for high risk phone codes
622992
56417
drop dupes
43193
import columbia regression dataset, has all info about complaints
43087


In [123]:
dc.to_csv(path + "test.csv", index=False)

### USE RISK SCORE TO DEVELOP LIST OF BUILDINGS UNKNOWN TO DOB, USING UPDATED COLUMBIA RISK SCORING 

In [2]:
import pandas as pd

path = "C:\\Users\\dmehri\\Documents\\DATA\\Building Census\\Columbia Data\\illegal conv\\"

dr = pd.read_csv(path + "IllegConv Ownership Risk Score 3.csv")

print len(dr)

#dr = dr[dr["Final Risk Category"] == "HIGH"].reset_index(drop=True)

dr["PhoneCode"] = dr["PhoneCode"].astype(str).map(str.strip)

print len(dr)

highphonecode = list(set(dr["PhoneCode"].tolist() ))

10717
10717


In [90]:
print "Calculate Risk Scores for Complaint/No Access and vios"

pd.set_option('chained_assignment', None)


dr["#ICComplaintsNoAccessVios"] = dr["#ICVios"] + dr["#ICNoAccess"]

dr["Risk Score CompNoAccessVios"] = 0.0

dr["Risk Category CompNoAccessVios"] = ""

#dr["ln_build_count"] = dr["Building Count"].log()

for i in range(0, len(dr)):
    #print dr["Building Count"][i]
    #dr["ln_build_count"][i] = np.log(dr["Building Count"][i])
    
    ln_bc = dr["ln_build_count"][i]
    
    dr["Risk Score CompNoAccessVios"][i] = dr["#ICComplaintsNoAccessVios"][i]/(ln_bc**2)
    
    if dr["Risk Score CompNoAccessVios"][i] > 1:
        dr["Risk Score CompNoAccessVios"][i] = 1
    
    dr["Risk Score CompNoAccessVios"][i] = (dr["Risk Score CompNoAccessVios"][i]*10).round(0)
    
    if dr["Risk Score CompNoAccessVios"][i] >= 8:
        dr["Risk Category CompNoAccessVios"][i] = "HIGH"
        
    elif (dr["Risk Score CompNoAccessVios"][i] >= 5 and dr["Risk Score CompNoAccessVios"][i] < 8):
        dr["Risk Category CompNoAccessVios"][i] = "MEDIUM"

    else:
        dr["Risk Category CompNoAccessVios"][i] = "LOW"


dr["Risk Score CompNoAccessVios"] = dr["Risk Score CompNoAccessVios"].astype(int)

Calculate Risk Scores for Complaint/No Access and vios


In [91]:
dr.to_csv(path + "test.csv", index=False)

### NA AND 1-2 FAMILY

In [92]:
pathc = "C:\\Users\\dmehri\\Documents\\DATA\\Building Census\\Columbia Data\\"

df = pd.read_csv(pathc + "Property BIN and Phone Number with Codes HPD COLUMBIA.csv")

df["BIN"] = df["BIN"].astype(str).map(str.strip)
df["PhoneCode"] = df["PhoneCode"].astype(str).map(str.strip)

df = df[["BIN", "PhoneCode"]]

print "len of df", len(df)

df2 = pd.read_csv(path + "Building Censes Illeg Conv Columbia Regression.csv")

df2 = df2[["BIN","NA", "1-2 FAMILY"]]

df2["1-2FandNA"] = df2["NA"] + df2["1-2 FAMILY"]

df2["1-2FandNA"] = df2["1-2FandNA"].astype(str)
df2["1-2FandNA"] = df2["1-2FandNA"].str.replace('1', '0')
df2["1-2FandNA"] = df2["1-2FandNA"].str.replace('2', '1')
df2["1-2FandNA"] = df2["1-2FandNA"].astype(float)

df2["BIN"] = df2["BIN"].astype(str).map(str.strip)

print "len of df2", len(df2)

df2Dic = df2.set_index('BIN')['1-2FandNA'].to_dict()

df["1_2FandNA"] = df["BIN"].map(df2Dic)


print "len of df", len(df)

df = df[["PhoneCode", "1_2FandNA"]]

dfG = df.groupby(['PhoneCode']).sum()
dfG = dfG.add_suffix('').reset_index()

dfG = dfG.sort_values(by = '1_2FandNA', ascending=False).reset_index(drop=True)

dfGDic = dfG.set_index('PhoneCode')['1_2FandNA'].to_dict()

print "map dictionary to dr"

dr["#1_2FandNA"] =  dr["PhoneCode"].map(dfGDic)



len of df 622992
len of df2 317538
len of df 622992
map dictionary to dr


In [87]:
dr.to_csv(path + "test.csv", index=False)

In [89]:
dr.dtypes

PhoneCode                              object
Building Count                          int64
#ICComplaints                         float64
#ICVios                               float64
#ICNoAccess                           float64
NA                                    float64
#ICComplaintsNoAccess                 float64
Building Count Category                object
ln_build_count                        float64
Risk Score IllegConv Complaints         int64
Risk Category IllegConv Complaints     object
Risk Score IllegConv Violations         int64
Risk Category IllegConv Violations     object
Risk Score NA                           int64
Risk Category NA                       object
Risk Score CompNoAccess                 int64
Risk Category CompNoAccess             object
Final Score                           float64
Final Risk Category                    object
#ICComplaintsNoAccessVios             float64
Risk Score CompNoAccessVios             int32
Risk Category CompNoAccessVios    

CREATE NA 1-2 FAMILY SCORE

In [93]:
#dr["#IC1-2FamNA"] = dr["#1-2 FAMILY"] + dr["NA"]

dr["Risk Score 1-2FamNA"] = 0.0

dr["Risk Category 1-2FamNA"] = ""


for i in range(0, len(dr)):
    ln_bc = dr["ln_build_count"][i]
    dr["Risk Score 1-2FamNA"][i] = dr["#1_2FandNA"][i]/(ln_bc**2)
    
    if dr["Risk Score 1-2FamNA"][i] > 1:
        dr["Risk Score 1-2FamNA"][i] = 1
    
    dr["Risk Score 1-2FamNA"][i] = (dr["Risk Score 1-2FamNA"][i]*10).round(0)
    
    if dr["Risk Score 1-2FamNA"][i] >= 8:
        dr["Risk Category 1-2FamNA"][i] = "HIGH"
        
    elif (dr["Risk Score 1-2FamNA"][i] >= 5 and dr["Risk Score 1-2FamNA"][i] < 8):
        dr["Risk Category 1-2FamNA"][i] = "MEDIUM"

    else:
        dr["Risk Category 1-2FamNA"][i] = "LOW"


dr["Risk Score 1-2FamNA"] = dr["Risk Score 1-2FamNA"].astype(int)



In [95]:
dr.to_csv(path + "test.csv", index=False)

In [96]:
dr["Final Score 2"] = (1.3*dr["Risk Score CompNoAccessVios"] + 
                      0.7*dr["Risk Score 1-2FamNA"])

dr["Final Score 2"] = (dr["Final Score 2"]/20.0)*10

dr["Final Risk Category 2"] = ""

for i in range(0, len(dr)):
    if dr["Final Score 2"][i] >= 8:
        dr["Final Risk Category 2"][i] = "HIGH"
    elif dr["Final Score 2"][i] < 5:
        dr["Final Risk Category 2"][i] = "LOW"
    else:
        dr["Final Risk Category 2"][i] = "MEDIUM"
       

                      


In [97]:
dr.to_csv(path + "test.csv", index=False)

In [46]:
dr.head(1)

,PhoneCode,Building Count,#ICComplaints,#ICVios,#ICNoAccess,NA,#ICComplaintsNoAccess,Building Count Category,ln_build_count,Risk Score IllegConv Complaints,...,Final Risk Category,#ICComplaintsNoAccessVios,Risk Score CompNoAccessVios,Risk Category CompNoAccessVios,#1-2 FAMILY,#IC1-2FamNA,Risk Score 1-2FamNA,Risk Category 1-2FamNA,Final Score 2,Final Risk Category 2
0,162639,2247,194.0,49.0,56.0,362.0,250.0,>100,7.717351,10,...,HIGH,299.0,10,HIGH,99.0,461.0,10,HIGH,10.0,HIGH


### GET THE ACTUAL BUILDINGS

In [98]:
print len(dr)
b = dr[dr["Final Risk Category 2"] == "HIGH"]
print len(b)

b = b["PhoneCode"].tolist()

print "get the list of buildings the owners own"

do = pd.read_csv(pathc + "Property BIN and Phone Number with Codes HPD COLUMBIA.csv")

print "len of do", len(do)
do["PhoneCode"] = do["PhoneCode"].astype(str)

do = do[do['PhoneCode'].isin(b)].reset_index(drop=True)

do["BIN"] = do["BIN"].astype(str)

print "len of do", len(do)

countbuild = set(do["BIN"].tolist())

print "number of buildings"
print len(countbuild)


df = pd.read_csv(path + "Building Censes Illeg Conv Columbia Regression.csv")
df["BIN"] = df["BIN"].astype(str)

binlist = list(set(do["BIN"].tolist()  ))

do2 = do[["BIN"]]

do2["High Risk"] = 1

do2Dic = do2.set_index('BIN')['High Risk'].to_dict()

df["High Risk"] = df["BIN"].map(do2Dic)
df["High Risk"] = df["High Risk"].fillna(0)





#df["High Risk"] = "NO"

#count = 0

"""
for i in range(0, len(df)):
    if df["BIN"][i] in binlist:
        df["High Risk"][i] = "YES" 
        count+=1
    
print "count", count

"""

10717
110
get the list of buildings the owners own
len of do 622992
len of do 15399
number of buildings
14459


'\nfor i in range(0, len(df)):\n    if df["BIN"][i] in binlist:\n        df["High Risk"][i] = "YES" \n        count+=1\n    \nprint "count", count\n\n'

In [71]:
df["BIN"][0]

'1000005'

In [100]:
df.to_csv(path + "IllegConv Ownership Risk Score 4.csv", index=False)

In [45]:
dr.to_csv(path + "test.csv", index=False)

In [36]:
dr.head(1)

,PhoneCode,Building Count,#ICComplaints,#ICVios,#ICNoAccess,NA,#ICComplaintsNoAccess,Building Count Category,ln_build_count,Risk Score IllegConv Complaints,...,Risk Score NA,Risk Category NA,Risk Score CompNoAccess,Risk Category CompNoAccess,Final Score,Final Risk Category,#ICComplaintsNoAccessVios,Risk Score CompNoAccessVios,Risk Category CompNoAccessVios,#1-2 FAMILY
0,162639,2247,194.0,49.0,56.0,362.0,250.0,>100,7.717351,10,...,10,HIGH,10,HIGH,10.0,HIGH,299.0,10,HIGH,99.0


# ********** OLD STUFF***************

### COLUMBIA REGRESSION ANALYSIS

In [29]:
path = "C:\\Users\\dmehri\\Documents\\DATA\\Building Census\\Columbia Data\\"

import pandas as pd

import numpy as np

df = pd.read_csv(path + "Comprehensive 1- 317K HPD List.csv")

df["BIN"] = df["BIN"].astype(str).map(str.strip)



### Complaints and Dispositions

In [30]:
pathi = "C:\\Users\\dmehri\\Documents\\DATA\\Building Census\\Columbia Data\illegal conv\\"

di = pd.read_csv(pathi + "DOB_Complaints_Received_Final_Disposition.csv", low_memory=False)


di["BIN"] = di["BIN"].astype(str).map(str.strip)


(1) ilc complaint with confirmed violation 

(2) ilc complaint and no access

(3) ilc complaint and BSC (affirmative disposition that not a violation)

(4) ilc complaint and all other dispositions, other than no access, vio and bsc

(5) No violation

In [31]:
pathb = "C:\\Users\\dmehri\\Documents\\DATA\\Building Census\\Columbia Data\\"

di = pd.read_csv(pathb + "DOB_Complaints_Received.csv",low_memory=False )

print "len of all complaints", len(di)

di["Complaint Category"] = di["Complaint Category"].astype(str).map(str.strip)

print "get only illegal conversion complaints, 45, 31, 71"

illegconv = ["45", "31", "71"]

di = di[di['Complaint Category'].isin(illegconv)].reset_index(drop=True)

print "len after getting only ics", len(di)


#print "get only tru complaints, drop I1, I2"

di["Disposition Code"] = di["Disposition Code"].astype(str).map(str.strip)

#ilist = ["I1", "I2"]

#di = di[~di['Disposition Code'].isin(ilist)].reset_index(drop=True)

#print "len after drop I1, I2", len(di)


di['Date Entered'] = pd.to_datetime(di['Date Entered'])

di["BIN"] = di["BIN"].astype(str).map(str.strip)

di = di.sort_values(by = 'Date Entered', ascending=True).reset_index(drop=True)

start_time = str(1) + '-' + str(1) + '-' + str(2017)
mask = di['Date Entered'] >= start_time
print "len before drop dates", len(di)
di = di.loc[mask].reset_index(drop=True)
print "len after drop", len(di)

print "Get only final complaints"

di = FinalComplaintDisposition(di)

#print len(di)

print "Data includes I1, and I2, complaint unsubstantiated"

print len(di)

print "Add number of complaints"

di2 = di.copy()

di2 = di2[["BIN"]]
di2["Count"] = 1
print "len di2", len(di2)
di2G = di2.groupby(['BIN']).sum()
di2G = di2G.add_suffix('').reset_index()

di2Dic =  di2G.set_index('BIN')['Count'].to_dict()
print "len di2Dic", len(di2Dic)

print "Map to df"

df["#ICComplaints"] = df["BIN"].map(di2Dic)
df["#ICComplaints"] = df["#ICComplaints"].fillna(0)


len of all complaints 2569490
get only illegal conversion complaints, 45, 31, 71
len after getting only ics 489177
len before drop dates 489177
len after drop 108688
Get only final complaints
len of datframe 108688
number of unique complaints 64946
0
3000
6000
9000
12000
15000
18000
21000
24000
27000
30000
33000
36000
39000
42000
45000
48000
51000
54000
57000
60000
63000
len of final dataframe 64946
Data includes I1, and I2, complaint unsubstantiated
64946
Add number of complaints
len di2 64946
len di2Dic 40355
Map to df


In [32]:
df.head(1)

,##,BIN,House #,Low House #,High House #,Boro,Street,CB,Block,Lot,...,SRO,DOF Stories,BIN Retired,SRO.1,BIN.2,Year Built,Built Code,Zoning Map,DataSet,#ICComplaints
0,268741,1000005,13,13,13,MANHATTAN,SOUTH STREET,101.0,4,7501,...,NaN,0,Active BIN,NaN,1000005,1969.0,NaN,12b,DataSet2 DataSet3,0.0


In [33]:
pd.set_option('chained_assignment', None)



dd2 = di.copy()

dd2 = dd2[["BIN","Status","Date Entered", "Disposition Code"]]

dd2["Disposition Code"] = dd2["Disposition Code"].astype(str).map(str.strip)

#codeCount = dd2[["Disposition Code"]]

#codeCount['Disposition Code'] = codeCount['Disposition Code'].str.replace(' ', 'nan')
#codes = set(dd2["Disposition Code"].tolist())

violist = ["A1", "A2", "A3", "A4", "A5", "A6", "A7", "A8", "A9",
          "B1", "B2", "H3", "H4", "H5", "L1", "L3", "Q1", "Q2",
          "Q3", "Q4", "Q5", "RU", "W1"]

noaccess = ["C1", "C2", "C3", "C4", "C5", "C6", "C7", "C8"]

noncomplaint = ["I1", "I2"]



print len(dd2)

dd2["Violation"] = 0
dd2["No Access"] = 0
dd2["Complaint Unsubstantiated"] = 0
count = 0
countnoaccess = 0
countnoncomplaint = 0
for i in range(0, len(dd2)):
    if i%8000==0:
        print i
    if dd2["Disposition Code"][i] in violist:
        dd2["Violation"][i] = 1
        count+=1
    if dd2["Disposition Code"][i] in noaccess:
        dd2["No Access"][i] = 1
        countnoaccess+=1
    if dd2["Disposition Code"][i] in noncomplaint:
        dd2["Complaint Unsubstantiated"][i] = 1
        countnoncomplaint+=1

#dd2G = dd2[["BIN","Violation", "No Access" ]]
dd2G = dd2[["BIN","Complaint Unsubstantiated", "Violation", "No Access" ]]
    
dd2G = dd2G.groupby(['BIN']).sum()
dd2G = dd2G.add_suffix('').reset_index()

#dd2G = dd2G.sort_values(by = 'Violation', ascending=False).reset_index(drop=True)

dd2DicVio = dd2G.set_index('BIN')['Violation'].to_dict()
dd2DicUns = dd2G.set_index('BIN')['Complaint Unsubstantiated'].to_dict()
dd2DicNoAccess = dd2G.set_index('BIN')['No Access'].to_dict()

df["#ICBSC"] = df["BIN"].map(dd2DicUns)
df["#ICBSC"] = df["#ICBSC"].fillna(0)

df["#ICVios"] = df["BIN"].map(dd2DicVio)
df["#ICVios"] = df["#ICVios"].fillna(0)

df["#ICNoAccess"] = df["BIN"].map(dd2DicNoAccess)
df["#ICNoAccess"] = df["#ICNoAccess"].fillna(0)

#print
#print "total number of dispositions", len(dd2)
#print "number of violations", count
#print "number no access", countnoaccess
print "number non complaints", countnoncomplaint

print "DONE"

64946
0
8000
16000
24000
32000
40000
48000
56000
64000
number non complaints 11368
DONE


### ADD NA FLAG

In [37]:
pd.set_option('chained_assignment', None)
dna = pd.read_csv(path + "Comprehensive 1- 317K HPD List.csv",low_memory=False )
dna["BIN"] = dna["BIN"].astype(str).map(str.strip)
dna["HPD Classifications"] = dna["HPD Classifications"].astype(str).map(str.strip).map(str.upper)

na = dna[dna["HPD Classifications"] == "NOT AVAILABLE"]

print len(na)

na["Count"] = 1

naDic =  na.set_index('BIN')['Count'].to_dict()

df["NA"] = df["BIN"].map(naDic)
df["NA"] = df["NA"].fillna(0)


123129


In [38]:
df.head(1)

,##,BIN,House #,Low House #,High House #,Boro,Street,CB,Block,Lot,...,BIN.2,Year Built,Built Code,Zoning Map,DataSet,#ICComplaints,#ICBSC,#ICVios,#ICNoAccess,NA
0,268741,1000005,13,13,13,MANHATTAN,SOUTH STREET,101.0,4,7501,...,1000005,1969.0,NaN,12b,DataSet2 DataSet3,0.0,0.0,0.0,0.0,1.0


In [24]:
di.to_csv(pathi  + "DOB_Complaints_Received_Final_Disposition_with_I1-2.csv", index=False)

In [34]:
df.to_csv(pathi + "test.csv", index=False)

In [36]:
df.head(1)

,##,BIN,House #,Low House #,High House #,Boro,Street,CB,Block,Lot,...,SRO.1,BIN.2,Year Built,Built Code,Zoning Map,DataSet,#ICComplaints,#ICBSC,#ICVios,#ICNoAccess
0,268741,1000005,13,13,13,MANHATTAN,SOUTH STREET,101.0,4,7501,...,NaN,1000005,1969.0,NaN,12b,DataSet2 DataSet3,0.0,0.0,0.0,0.0


ADD BUILDING CLASSFICATIONS

In [47]:

df["Class Desc"] = df["Class Desc"].astype(str).map(str.strip).map(str.upper)


df['Class Desc'] = df['Class Desc'].str.replace('RESIDENCE-MULTI USE:1 FAMILY W/STORES OROFFICE','RESIDENCE-MULTI USE 1-2 FAMILY')
df['Class Desc'] = df['Class Desc'].str.replace('RESIDENCE-MULTI USE:PRIMARY 1 FAMILY W STORE/OFFICE','RESIDENCE-MULTI USE 1-2 FAMILY')
df['Class Desc'] = df['Class Desc'].str.replace('RESIDENCE-MULTI USE:PRIMARY 2 FAMILY W STORE/OFFICE','RESIDENCE-MULTI USE 1-2 FAMILY')

df['Class Desc'] = df['Class Desc'].str.replace('RESIDENCE-MULTI USE:PRIMARY 3 FAMILY W STORE/OFFICE','RESIDENCE-MULTI USE 3-6 FAMILY')
df['Class Desc'] = df['Class Desc'].str.replace('RESIDENCE-MULTI USE:PRIMARY 4 FAMILY W STORE/OFFICE','RESIDENCE-MULTI USE 3-6 FAMILY')
df['Class Desc'] = df['Class Desc'].str.replace('RESIDENCE-MULTI USE:PRIMARY 5/6 FAM W STORE/OFFICE','RESIDENCE-MULTI USE 3-6 FAMILY')


df["Class Desc ReCode"] = df['Class Desc'].str.split(":").str[0]

df['Class Desc ReCode'] = df['Class Desc ReCode'].str.replace('1 FAMILY DWELLINGS', '1-2 FAMILY')
df['Class Desc ReCode'] = df['Class Desc ReCode'].str.replace('2 FAMILY DWELLINGS', '1-2 FAMILY')
#df['Class Desc ReCode'] = df['Class Desc ReCode'].str.replace('RESIDENCE-MULTI USE', 'RESIDENCE-MULTI USE 1-6 FAMILY')

classes = sorted( list(set(df["Class Desc ReCode"].tolist() )) )
print len(classes)


print "label 1-2 families"

df["1-2 FAMILY"] = 0

for i in range(0, len(df)):
    if "1-2 FAMILY" in df['Class Desc ReCode'][i]:
        df["1-2 FAMILY"][i] = 1
        
print "DONE"

47
label 1-2 families


### ADD MULTIPLE DWELLING

In [55]:
dm = pd.read_csv(path + "Multiple_Dwelling_Registrations.csv", low_memory=False)

In [61]:
dm["BIN"] = dm["BIN"].astype(str)
dm["BIN"] = dm["BIN"].str.split(".").str[0]
dm["BIN"] = dm["BIN"].map(str.strip)

binList = dm["BIN"].tolist()

dm = dm[["BIN"]]
dm["MD"] = 1

dmDic = dm.set_index('BIN')['MD'].to_dict()

df["MD"] = df["BIN"].map(dmDic)
df["MD"] = df["MD"].fillna(0)

        

### ADD PORTFOLIO SIZE

In [64]:
pathi

'C:\\Users\\dmehri\\Documents\\DATA\\Building Census\\Columbia Data\\illegal conv\\'

In [76]:
dp = pd.read_csv(pathi + "IllegConv Ownership Risk Score 3.csv")

dp2 = dp[["PhoneCode", "Building Count"]]

dp2.head(1)

dp = pd.read_csv(path + "Property BIN and Phone Number with Codes HPD COLUMBIA.csv")

dp = dp[["BIN", "PhoneCode"]]

dp = pd.merge(dp, dp2, how='left', on=['PhoneCode'])



In [80]:
df.head(1)

,##,BIN,House #,Low House #,High House #,Boro,Street,CB,Block,Lot,...,Zoning Map,DataSet,#ICComplaints,#ICBSC,#ICVios,#ICNoAccess,NA,Class Desc ReCode,1-2 FAMILY,MD
0,268741,1000005,13,13,13,MANHATTAN,SOUTH STREET,101.0,4,7501,...,12b,DataSet2 DataSet3,0.0,0.0,0.0,0.0,1.0,CONDOMINIUMS,0,0.0


In [87]:
dp2 = dp[["BIN", "Building Count"]]

dp2["BIN"] = dp2["BIN"].astype(str)

dpG1 = (dp2.groupby('BIN').agg({'Building Count':'mean'}).reset_index().rename(columns={'Building Count':'Owner Portfolio Size Mean'}) )

print len(df)
df2 = pd.merge(df, dpG1, how='left', on=['BIN'])
print len(df2)


317538
317538


In [86]:
dpG1["BIN"][0]

'1000005'

In [91]:
df.to_csv(pathi + "Building Censes Illeg Conv Columbia Regression.csv", index=False)

In [89]:
df.head(1)

,##,BIN,House #,Low House #,High House #,Boro,Street,CB,Block,Lot,...,Zoning Map,DataSet,#ICComplaints,#ICBSC,#ICVios,#ICNoAccess,NA,Class Desc ReCode,1-2 FAMILY,MD
0,268741,1000005,13,13,13,MANHATTAN,SOUTH STREET,101.0,4,7501,...,12b,DataSet2 DataSet3,0.0,0.0,0.0,0.0,1.0,CONDOMINIUMS,0,0.0


In [ ]:
d3.to_csv(path + "DOF Building Classifications Recoded.csv", index=False)

In [ ]:
d2.to_csv(path + "test.csv", index=False)

In [ ]:
df.head(1)

MERGE BUILDING CLASSIFICAITON

In [ ]:
d2merge = d2[["BIN", "Class Desc ReCode"]]
d2merge = d2merge.rename(columns={'Class Desc ReCode': 
                                  'Building Classification'})

d2mergeDic = d2merge.set_index('BIN')['Building Classification'].to_dict()



df["Building Classification"] = df["BIN"].map(d2mergeDic)


In [ ]:
df.head(1)

In [ ]:
df.to_csv(path + "Building Census Illegal Conversion NA Analysis.csv", index=False)

CALCULATE WITHIN CLASSIFICAITION

In [ ]:
pd.set_option('chained_assignment', None)

dc = pd.read_csv(path + "Comprehensive 1- 317K HPD List.csv",low_memory=False )

dc["BIN"] = dc["BIN"].astype(str).map(str.strip)


d2 = dc[["BIN", "Class Desc"]]

d2["Class Desc"] = d2["Class Desc"].astype(str).map(str.strip).map(str.upper)


d2['Class Desc'] = d2['Class Desc'].str.replace('RESIDENCE-MULTI USE:1 FAMILY W/STORES OROFFICE','RESIDENCE-MULTI USE 1-2 FAMILY')
d2['Class Desc'] = d2['Class Desc'].str.replace('RESIDENCE-MULTI USE:PRIMARY 1 FAMILY W STORE/OFFICE','RESIDENCE-MULTI USE 1-2 FAMILY')
d2['Class Desc'] = d2['Class Desc'].str.replace('RESIDENCE-MULTI USE:PRIMARY 2 FAMILY W STORE/OFFICE','RESIDENCE-MULTI USE 1-2 FAMILY')

d2['Class Desc'] = d2['Class Desc'].str.replace('RESIDENCE-MULTI USE:PRIMARY 3 FAMILY W STORE/OFFICE','RESIDENCE-MULTI USE 3-6 FAMILY')
d2['Class Desc'] = d2['Class Desc'].str.replace('RESIDENCE-MULTI USE:PRIMARY 4 FAMILY W STORE/OFFICE','RESIDENCE-MULTI USE 3-6 FAMILY')
d2['Class Desc'] = d2['Class Desc'].str.replace('RESIDENCE-MULTI USE:PRIMARY 5/6 FAM W STORE/OFFICE','RESIDENCE-MULTI USE 3-6 FAMILY')


d2["Class Desc ReCode"] = d2['Class Desc'].str.split(":").str[0]

d2['Class Desc ReCode'] = d2['Class Desc ReCode'].str.replace('1 FAMILY DWELLINGS', '1-2 FAMILY')
d2['Class Desc ReCode'] = d2['Class Desc ReCode'].str.replace('2 FAMILY DWELLINGS', '1-2 FAMILY')
#d2['Class Desc ReCode'] = d2['Class Desc ReCode'].str.replace('RESIDENCE-MULTI USE', 'RESIDENCE-MULTI USE 1-6 FAMILY')

classes = sorted( list(set(d2["Class Desc ReCode"].tolist() )) )

d2["IllegConv"] = d2["BIN"].map(di2Dic)
d2["IllegConv"] = d2["IllegConv"].fillna(0)


d2["NA"] = d2["BIN"].map(naDic)
d2["NA"] = d2["NA"].fillna(0)

In [ ]:
d2.to_csv(path + "test.csv", index=False)

In [ ]:
d2["Not NA"] = 0
for i in range(0, len(d2)):
    if d2["NA"][i] == 0:
        d2["Not NA"][i] = 1
        

In [ ]:
d2.tail(5)

In [ ]:
d2 = d2.rename(columns={'Class Desc ReCode': 'Building Classification'})

d2["Building Count"] = 1

dclass = d2[["Building Classification", "Building Count", 
             "IllegConv", "NA"]]

dcount = d2[["Building Classification", "Building Count", "NA", "Not NA"]]

dcount = dcount.groupby(['Building Classification']).sum()

dcount = dcount.add_suffix('').reset_index()

dcount = dcount.sort_values(by = 'Building Count', ascending=False).reset_index(drop=True)

dclassNA = dclass[dclass["NA"] == 1]

dclassNA = dclassNA[["Building Classification", "IllegConv"]]

print "Number of NAs", len(dclassNA)

dclassNA = dclassNA.groupby(['Building Classification']).sum()

dclassNA = dclassNA.add_suffix('').reset_index()

dcount2 = pd.merge(dcount, dclassNA, how='left', on=['Building Classification'])

dcount2 = dcount2.rename(columns={'IllegConv': 'IllegConvNA'})

dclassNA = dclass[dclass["NA"] == 0]

dclassNA = dclassNA[["Building Classification", "IllegConv"]]

print "Number of Not NA", len(dclassNA)

dclassNA = dclassNA.groupby(['Building Classification']).sum()

dclassNA = dclassNA.add_suffix('').reset_index()

dcount3 = pd.merge(dcount2, dclassNA, how='left', on=['Building Classification'])

dcount3 = dcount3.rename(columns={'IllegConv': 'IllegConvNotNA'})


In [ ]:
dcount3.to_csv(path + "Build Class IllegConv Conditioned NA.csv", index=False)

In [ ]:
len(dclassNA)